In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
training_examples=10000
testing_examples=1000
#sin函数的采样间隔
sample_gap=0.01
timesteps=20

In [3]:
#处理数据 数据是20*1
def generate_data(seq,timesteps):
    x=[]
    y=[]
    for i in range(len(seq)-timesteps-1):
        x.append(seq[i:i+timesteps])#20个为一组 是二维
        y.append(seq[i+timesteps])#一维数组就是第21个被预测的值
    return np.array(x,dtype=np.float32),np.array(y,dtype=np.float32)

test_start=training_examples*sample_gap
#10000*0.01=100
test_end=test_start+testing_examples*sample_gap
#100+1000*0.01=110
train_x,train_y=generate_data(np.sin(np.linspace(0,test_start,training_examples)),timesteps)
#0--100之间 生成10000个数字  投放到sin带入生成器中
test_x,test_y=generate_data(np.sin(np.linspace(test_start,test_end,testing_examples)),timesteps)
print(train_x.shape)
print(train_y.shape)

(9979, 20)
(9979,)


In [4]:
#如题
#def get_batches(x,y,batch_size):
    #for i in range(0,len(x),batch_size):
        #begin_i=i
        #end_i=i+batch_size if (i+batch_size)<len(x) else len(x)
        #yield x[begin_i:end_i,:],y[begin_i:end_i]   

In [5]:
def get_batches(x,y,batch_size):
    for i in range(0,len(x),batch_size):
        if (i+batch_size)<len(x):
            begin_i=i
            end_i=i+batch_size 
            yield x[begin_i:end_i,:],y[begin_i:end_i]  

In [6]:
#输入，输出，以及丢弃率，的占位符  x=[64,20] 后面的1是因为数据只是一个
def build_inputs(timesteps):
    x=tf.placeholder(tf.float32,[None,timesteps,1],name='input_x')
    y=tf.placeholder(tf.float32,[None,1],name='input_y')
    keep_prob=tf.placeholder(tf.float32,name='keep_prob')
    return x,y,keep_prob

In [7]:
#lstm  
def build_lstm(lstm_size,num_layers,batch_size,keep_prob):#batch_size:num_seqs * num_steps
    def build_cell(lstm_size,keep_prob):
        lstm=tf.contrib.rnn.BasicLSTMCell(lstm_size)
        drop=tf.contrib.rnn.DropoutWrapper(lstm,output_keep_prob=keep_prob)
        return drop
    cell=tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size,keep_prob) for _ in range(num_layers)])
    initial_state=cell.zero_state(batch_size,tf.float32)
    return cell,initial_state

In [8]:
#输出
def build_output(lstm_output):
    print('lstm_output=',lstm_output.shape)
    outputs=lstm_output[:,-1,:]#取最后一维
    print('outputs=',outputs.shape)
    predictions=tf.contrib.layers.fully_connected(outputs,1,activation_fn=tf.tanh)
    return predictions    

In [9]:
#损失和优化
def build_loss(predictions,y):
    loss=tf.losses.mean_squared_error(y,predictions)
    return loss
def build_optimizer(loss):
    optimizer=tf.train.AdamOptimizer().minimize(loss)
    return optimizer  

In [10]:
#定义类
class charRNN:
    def __init__(self,batch_size=64,lstm_size=20,timesteps=20,num_layers=2):
        #输入
        self.inputs,self.targets,self.keep_prob=build_inputs(num_steps)
        print(self.inputs.shape)
        
        #lstm
        cell,self.initial_state=build_lstm(lstm_size,num_layers,batch_size,self.keep_prob)
        
        #运行
        lstm_output,state=tf.nn.dynamic_rnn(cell,self.inputs,initial_state=self.initial_state)
        self.final_state=state
        
        #预测结果
        self.prediction=build_output(lstm_output)
        
        #loss 和 optimizer
        print(self.prediction.shape,self.targets.shape)
        self.loss=build_loss(self.prediction,self.targets)
        self.optimizer=build_optimizer(self.loss)

In [11]:
batch_size=64
lstm_size=20
num_layers=2
keep_prob=0.8
num_steps=20
epochs=50
save_every_n=10

In [12]:
model=charRNN(batch_size,lstm_size,timesteps,num_layers)
#saver=tf.train.Saver(max_to_keep=100)
session=tf.Session()
with session.as_default() as sess:
    sess.run(tf.global_variables_initializer())
    count=0
    for e in range(epochs):
        new_state=sess.run(model.initial_state)
        loss=0
        for x,y in get_batches(train_x,train_y,batch_size):
            count+=1
            #print(x.shape,y.shape)
            feed={model.inputs:x[:,:,None],
                  model.targets:y[:,None],
                  model.keep_prob:keep_prob,
                  model.initial_state:new_state}
            prediction_,batch_loss,new_state,_=sess.run([model.prediction,model.loss,model.final_state,model.optimizer],feed_dict=feed)
            #print( 'prediction_:',np.shape(prediction_))
        if e%5==0:
            print('轮数：{}/{}...'.format(e+1,epochs),
                  '训练步数：{}...'.format(count),
                  '训练误差：{:.4f}...'.format(batch_loss)
                 )
            #if (count % save_every_n==0):
                #saver.save(sess,'checkpoints/i{}.ckpt'.format(count,lstm_size))
    #saver.save(sess,'checkpoints/i{}.ckpt'.format(count,lstm_size))

(?, 20, 1)
lstm_output= (64, 20, 20)
outputs= (64, 20)
(64, 1) (?, 1)
轮数：1/50... 训练步数：155... 训练误差：0.0323...
轮数：6/50... 训练步数：930... 训练误差：0.0018...
轮数：11/50... 训练步数：1705... 训练误差：0.0007...
轮数：16/50... 训练步数：2480... 训练误差：0.0007...
轮数：21/50... 训练步数：3255... 训练误差：0.0005...
轮数：26/50... 训练步数：4030... 训练误差：0.0006...
轮数：31/50... 训练步数：4805... 训练误差：0.0004...
轮数：36/50... 训练步数：5580... 训练误差：0.0004...
轮数：41/50... 训练步数：6355... 训练误差：0.0004...
轮数：46/50... 训练步数：7130... 训练误差：0.0003...


In [15]:
keep_prob=1
with session.as_default() as sess:
    #结果
    new_state=sess.run(model.initial_state)
    feed={model.inputs:test_x[:,:,None],
          model.keep_prob:keep_prob,
          model.initial_state:new_state}
    results=sess.run(model.prediction,feed_dict=feed)
    plt.plot(results,'r',label='predicted')
    plt.plot(test_y,'g-',label='real sin')
    plt.legend()
    plt.show()

InvalidArgumentError: ConcatOp : Dimensions of inputs should match: shape[0] = [979,1] vs. shape[1] = [64,20]
	 [[Node: rnn/while/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/concat = ConcatV2[N=2, T=DT_FLOAT, Tidx=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](rnn/while/TensorArrayReadV3, rnn/while/Identity_4, rnn/while/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/concat/axis)]]

Caused by op 'rnn/while/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/concat', defined at:
  File "D:\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\Anaconda3\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "D:\Anaconda3\lib\site-packages\traitlets\config\application.py", line 653, in launch_instance
    app.start()
  File "D:\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 474, in start
    ioloop.IOLoop.ins